In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

/Users/jteruya/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Equity Class

Class Attributes include:
- name
- file
- df

Class Methods include:
- historical_volatility (The calculation was taken from [here](https://www.investopedia.com/ask/answers/021015/how-can-you-calculate-volatility-excel.asp))
- min_max_scale

Class Helper Methods include:
- _date_fields

In [2]:
class equity():
    
    def __init__(self, name, file):
        self.name = name
        self.file = file
        self.df = pd.read_csv(file)
        self._date_field('Date')
        self.df.set_index(['Date'], inplace=True)
    
    def _date_field(self, field_name):
        self.df[field_name] = pd.to_datetime(self.df[field_name])
        return self
    
    def historical_volatility(self, lookback_in_days):
        def _return_on_price(current_price, previous_price):
            return (current_price/previous_price) - 1
        self.df['Adj Close Shift'] = self.df['Adj Close'].shift(1)
        self.df['Adj Close Price Return'] = self.df.apply(lambda row : _return_on_price(row['Adj Close'], row['Adj Close Shift']), axis = 1)
        self.df['Rolling Volatility'] = np.sqrt(lookback_in_days) * self.df['Adj Close Price Return'].rolling(lookback_in_days).std() * 100
        return self
    
    def min_max_scale(self, field_name):
        min_max_scaler = preprocessing.MinMaxScaler()
        self.df[field_name + ' Scaled'] = min_max_scaler.fit_transform(self.df[[field_name]])
        return self